<a href="https://colab.research.google.com/github/hyguozz/DNAm-Aging-clock/blob/main/FL_DNAm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip3 install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 14.1 MB/s 


In [ ]:

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K



In [ ]:
infile = open('/content/drive/MyDrive/Aging/data/DNAm_part1_typeII_2838.pkl','rb')
DNAm_data= pickle.load(infile)
infile.close()
imputer = KNNImputer(n_neighbors=10)
DNAm_data = pd.DataFrame(imputer.fit_transform(DNAm_data),columns = DNAm_data.columns)

In [ ]:
DNAm_data.shape

(2838, 8193)

In [ ]:
# May 30, 2022
# Good
#infile = open('/content/drive/MyDrive/Aging/TrainDatasetI17.pkl','rb')
infile = open('/content/drive/MyDrive/Aging/TrainDatasetI530.pkl','rb')
DNAm_data_train= pickle.load(infile)
#DNAm_data_train = DNAm_data_train[DNAm_data_train['age']<100]
train = DNAm_data_train.copy()

df_age01 = train[train['age']<=0].copy()

df_age01 = df_age01.sample(n = 100)
train = train[train['age']>0].copy().append(df_age01)
train.loc[train['dataset']=='GSE59157','age']= train.loc[train['dataset']=='GSE59157','age']/12

#infile = open('/content/drive/MyDrive/Aging/TestDatasetI17.pkl','rb')
infile = open('/content/drive/MyDrive/Aging/TestDatasetI530.pkl','rb')

DNAm_data= pickle.load(infile)
#DNAm_data = DNAm_data[DNAm_data['age']<100]
test = DNAm_data.copy()
df_age0 = test[test['age']<=0].copy()
df_age0 = df_age0.sample(n = 40)
test = test[test['age']>0].copy().append(df_age0)
test.loc[test['dataset']=='GSE59157','age']= test.loc[test['dataset']=='GSE59157','age']/12
test, val_data = train_test_split(test, test_size=0.10)

In [ ]:
# May 16, 2022

infile = open('/content/drive/MyDrive/Aging/TrainDatasetI17.pkl','rb')
DNAm_data_train= pickle.load(infile)
DNAm_data_train = DNAm_data_train[DNAm_data_train['age']<100]
train = DNAm_data_train.copy()

infile = open('/content/drive/MyDrive/Aging/TestDatasetI17.pkl','rb')
DNAm_data= pickle.load(infile)
DNAm_data = DNAm_data[DNAm_data['age']<100]
test = DNAm_data.copy()
#test, val_data = train_test_split(test, test_size=0.10)

In [ ]:
# Scaled data 
# May 17, 2022
X_train = train.iloc[:,3:].to_numpy()
transformer = RobustScaler().fit(X_train)
X_train = transformer.transform(X_train)

X_test = test.iloc[:,3:].to_numpy()
transformer = RobustScaler().fit(X_test)
X_test = transformer.transform(X_test)

y_train = train['age'].astype('float')
y_test = test['age'].astype('float')

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]

    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    #print(local_count)
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    #print(steps)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(DNAm_X, 
                                                    DNAm_age, 
                                                    test_size=0.3, 
                                                    random_state=42)

X_train = list(X_train.to_numpy())
y_train = list(y_train.to_numpy())
X_test = list(X_test.to_numpy())
y_test = list(y_test.to_numpy())

In [ ]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 


In [ ]:
#create clients
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [ ]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))


In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape):
        model = Sequential()
        model.add(Dense(32, input_shape=(shape,)))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(1))

        return model

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape):
        model = Sequential()
        model.add(Dense(128, input_shape=(shape,)))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(128))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(64))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(64))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(32))
        model.add(BatchNormalization())
        model.add(Activation('LeakyReLU'))
        model.add(Dense(1))

        return model

In [ ]:

def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.MeanAbsoluteError()
    pred1 = model.predict(X_test).flatten()
    loss11 = cce(Y_test, pred1)
    #print(X_test)
    #print(loss11)
    cce2 = tf.keras.losses.MeanSquaredError()
    loss22 = cce2(Y_test, pred1)
    meae = median_absolute_error(Y_test, pred1)
    #acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | MedianAE: {:.3} | MAE: {:.3} | MSE: {:.3}'.format(comm_round,  meae, loss11,loss22))
    return loss11,loss22


In [ ]:
cce = tf.keras.losses.MeanAbsoluteError()
pred1 = global_model.predict(X_test).flatten()
#pred1 = pred1.numpy()
loss11 = cce(Y_test, pred1)
#print(X_test)
#print(loss11)
cce2 = tf.keras.losses.MeanSquaredError()
loss22 = cce2(Y_test, pred1)
#acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
print('comm_round: {} | MAE: {:.3%} | MSE: {}'.format(comm_round,  loss11,loss22))

comm_round: 49 | MAE: 466.637% | MSE: 44.436431884765625


# Centralized Data 
# June 1st, 2022

In [ ]:
# Centralized data
# May 17, 2022

X_train = train.iloc[:,3:].to_numpy()
transformer = RobustScaler().fit(X_train)
X_train = transformer.transform(X_train)

X_test = test.iloc[:,3:].to_numpy()
transformer = RobustScaler().fit(X_test)
X_test = transformer.transform(X_test)

y_train = train['age'].astype('float')
y_test = test['age'].astype('float')

In [ ]:
# MLP traing and testing, Centralized data
# May 13rd, 2022
# Part I
smlp_global = SimpleMLP()
model = smlp_global.build(feature_size)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
model.fit(X_train, y_train, epochs = 200, verbose = 0)


In [ ]:
# MLP traing and testing, Centralized data
# May 13rd, 2022
# Part II

cce = tf.keras.losses.MeanAbsoluteError()
pred1 = model.predict(X_test).flatten()
loss11 = cce(y_test, pred1)
cce2 = tf.keras.losses.MeanSquaredError()
loss22 = cce2(y_test, pred1)
meae = median_absolute_error(y_test, pred1)
#acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
print('MedianAE: {:.3} | MAE: {:.3} | MSE: {:.3}'.format( meae, loss11,loss22))


MedianAE: 2.41 | MAE: 3.9 | MSE: 39.6


In [ ]:
lr = 0.001 
comms_round = 50
loss='mean_squared_error'
metrics=[tf.keras.metrics.MeanAbsoluteError()]
optimizer = tf.keras.optimizers.Adam(learning_rate=lr, decay = lr/comms_round)
#optimizer = SGD(learning_rate=lr, decay=lr / comms_round, momentum=0.9)  


In [ ]:
smlp_global = SimpleMLP()
global_model = smlp_global.build(feature_size)


In [ ]:
#initialize global model
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(feature_size)
        #local_model.compile(loss=loss, 
        #              optimizer=optimizer, 
        #              metrics=metrics)
        local_model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=3, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_MAE, global_MSE = test_model(X_test, Y_test, global_model, comm_round)


comm_round: 0 | MedianAE: 3.49 | MAE: 4.73 | MSE: 45.5
comm_round: 1 | MedianAE: 3.38 | MAE: 4.6 | MSE: 43.0
comm_round: 2 | MedianAE: 3.35 | MAE: 4.62 | MSE: 44.2
comm_round: 3 | MedianAE: 3.47 | MAE: 4.75 | MSE: 46.7
comm_round: 4 | MedianAE: 3.28 | MAE: 4.54 | MSE: 43.5
comm_round: 5 | MedianAE: 3.52 | MAE: 4.76 | MSE: 46.4
comm_round: 6 | MedianAE: 3.45 | MAE: 4.65 | MSE: 45.0
comm_round: 7 | MedianAE: 3.4 | MAE: 4.6 | MSE: 43.5
comm_round: 8 | MedianAE: 3.38 | MAE: 4.64 | MSE: 45.4
comm_round: 9 | MedianAE: 3.49 | MAE: 4.7 | MSE: 44.1
comm_round: 10 | MedianAE: 3.41 | MAE: 4.59 | MSE: 42.5
comm_round: 11 | MedianAE: 3.32 | MAE: 4.52 | MSE: 41.9
comm_round: 12 | MedianAE: 3.47 | MAE: 4.65 | MSE: 43.4
comm_round: 13 | MedianAE: 3.16 | MAE: 4.42 | MSE: 41.4
comm_round: 14 | MedianAE: 3.28 | MAE: 4.49 | MSE: 42.0
comm_round: 15 | MedianAE: 3.39 | MAE: 4.59 | MSE: 42.5
comm_round: 16 | MedianAE: 3.24 | MAE: 4.45 | MSE: 42.2
comm_round: 17 | MedianAE: 3.27 | MAE: 4.49 | MSE: 42.5
comm_r